# Homework 2

For this homework, you'll re-write the `Parser` class.  Instead of using keyword based parsing, as was done in old-school text adventure games, we'll see how we can use large language models (LLMs) like GPT.  This will allow us to do two important things:

1. We can use the LLM to generate more evocative descriptions.
2. We can do more flexible matching of user commands.

## Setting up the OpenAI API

You'll need to run `python -m pip install openai` in your terminal, and specify your OpenAI API Key (which you can find [here](https://platform.openai.com/api-keys)) on the terminal with the command `export OPENAI_API_KEY=sk-##########`. If you're using VS Code, you can launch it from your terminal with the `code` command and prefix it with this export statement on the same line.  For example:

```
cd your/homework/dir
source venv/bin/activate
OPENAI_API_KEY=sk-##########  code .
```


In [1]:

from openai import OpenAI

import os
from getpass import getpass

if 'OPENAI_API_KEY' not in os.environ:
    print("You didn't set your OPENAI_API_KEY on the command line.")
    os.environ['OPENAI_API_KEY'] = getpass("Please enter your OpenAI API Key: ")


You didn't set your OPENAI_API_KEY on the command line.


## Your first call to the OpenAI API



In [2]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are the narrator for a text adventure game. You create short, evocative descriptions of the scenes in the game. Use a style like Jane Austen."
    },
    {
      "role": "user",
      "content": "look"
    },
    {
      "role": "assistant",
      "content": "You are standing in a small cottage.\nExits:\nOut to Garden Path\n\nYou see:\n * a fishing pole\n * a poisonous potion\n"
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [3]:
import textwrap
def wrap_text(text: str, width: int = 80) -> str:
    """
    Keeps text output narrow enough to easily be read
    """
    lines = text.split("\n")
    wrapped_lines = [textwrap.fill(line, width) for line in lines]
    return "\n".join(wrapped_lines)

In [4]:
print(wrap_text(response.choices[0].message.content))

You find yourself in a charming garden, with colorful flowers blooming all
around.
Exits:
Back to Cottage

You see:
 * a rusted key
 * a mysterious stone statue


## Environment

Next, let's import our text adventure games. 

In [4]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

### Part 1: Using GPT to Write Descriptions

The text of our adventures in HW1 was a bit bland.  We will can make it more evocative with GPT by subclassing the game's default `Parser` class and having our new `GptParser` subclass override the `ok` and `fail` methods that describe things to the player.

In [6]:
import json
import tiktoken
import numpy as np

class GptParser(parsing.Parser):
    def __init__(self, game, echo_commands=True, verbose=False):
        super().__init__(game, echo_commands=echo_commands)
        self.verbose=verbose
        self.client = OpenAI()
        self.gpt_model = "gpt-4"
        # self.gpt_model = "gpt-3.5-turbo"
        self.max_output_tokens = 256 # You get to pick this
        self.max_tokens = 8192-self.max_output_tokens # GPT-4's max total tokens
        # self.max_tokens = 4096-self.max_output_tokens # GPT-3.5-turbo's max total tokens
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        self.system_instructions = "You are the narrator for a text adventure game. You create short, evocative descriptions of the scenes in the game. Include descriptions of the items and exits available to the Player. Use a style like Edgar Allan Poe."
        self.encode = tiktoken.get_encoding("cl100k_base")
        self.sys_inst_len = len(self.encode.encode(self.system_instructions))
        # make room for the system prompt that we always pass in
        self.max_tokens -= self.sys_inst_len

    def gpt_describe(self, system_instructions, command_history):
        """
        Generate a description with GPT.  This takes two arguments:
        * The system instructions, which is the prompt that describes 
          how you'd like GPT to behave.
        * The command history - this is a list of previous user input 
          commands and game descriptions. It's given as context to GPT.
        The Parser class manages the command_history via the 
        `add_command_to_history` and `add_description_to_history` functions
        which use the ChatGPT format with commands being assigned role: user,
        and descriptions being assigned role: assistant.
        """
        try:
            messages = [{
                "role": "system",
                "content": system_instructions
            }]
            context = self.limit_context_length(command_history, self.max_tokens)
            messages.extend(context)
            if self.verbose:
                print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model=self.gpt_model,
                messages=messages,
                temperature=1,
                max_tokens=self.max_output_tokens,
                top_p=0.5,
                frequency_penalty=0,
                presence_penalty=0
            )
            content = response.choices[0].message.content
            return content
        except Exception as e:
            return f"Something went wrong with GPT: {e}"
        
    def limit_context_length(self, command_history, max_tokens, max_turns=1000):
        """
        This method limits the length of the command_history 
        to be less than the max_tokens and less than max_turns. 
        The least recent messages are disregarded from the context. 
        This function is non-destructive and doesn't modify command_history.
        """
        if len(command_history) > max_turns:
            adj_command_history = command_history[-max_turns:]
        else:
            adj_command_history = command_history[:]

        # get each message's context
        commands = [i['content'] for i in adj_command_history]

        # based on the cookbook from OpenAI GitHub link CCB provided
        tokens_per_message=3
        command_lengths = [len(self.encode.encode(i))+tokens_per_message for i in commands]

        command_lengths.reverse()
        cum_list_lengths = np.cumsum(command_lengths)
        for index, i in enumerate(cum_list_lengths):
            # we add 3 because every reply is primed with <|start|>assistant<|message|> worth 3 more tokens
            # this is also based on the CCB ED link
            if i + 3 > max_tokens:
                # if the command history is exceeded, we return index places from the end
                # for example if index 3 (the 4th value in the cumsum since these start with index 0)
                # exceeds the max_tokens, but index 2 (3rd value in the cumsum) didn't, then
                # slicing from [-3:] returns fewer than the max_tokens
                return adj_command_history[-index:]
        # in the case that the adjusted command history is shorter than the max_tokens
        return adj_command_history[:] # TODO - use tiktoken return a limited history

    def ok(self, description):
        """
        In this homework, we'll replace this with a call to the OpenAI API
        in order to create more evocative descriptions.  For this part, 
        all you need to do is create your own system instructions.   
        
        You should: 
        1. Design system_instructions to pass to GPT to narrate the game based
           on the user's command, and the game's default response. 
        2. Query GPT using your gpt_describe and self.command_history
        3. Print GPT's output (optionally using self.wrap_text)
        4. Add GPT's output to the command history using add_description_to_history.
        """
        self.command_history.append({"role": "assistant", "content": description})
        response = self.gpt_describe(self.system_instructions, self.command_history)
        self.add_description_to_history(response)
        print(wrap_text(response) + '\n')

    def fail(self, description):
        """
        In this homework, we'll replace this with a call to the OpenAI API
        in order to create more useful error messages descriptions.  You should
        do the same steps as for the ok() command, but optionally customize 
        the system_instructions to tell the user that their command failed.
        """
        self.command_history.append({"role": "assistant", "content": description})
        print("COMMAND FAILED")
        
        system_instructions = "You are the narrator for a text adventure game. \
The player entered a command that failed in the game. \
Try to help the player understand why the command failed."
        
        response = self.gpt_describe(system_instructions, self.command_history)
        if self.verbose:
            print("GPT's Error Description:")
        # we chose to leave out this addition to the description, as fail cases often had hallucinations,
        # and we didn't want this to become a part of the history, which would be given as a context
        # at the next call
        # self.add_description_to_history(response)
        print(wrap_text(response) + '\n')

    

### Describing Action Castle

Let's use our new system prompt to have GPT generate descriptions of Action Castle!  We'll import my implementation of the Action Castle game, and then run the game with your GptParser.

The below run made it all the way to end the of game. The player just needed to go east the throne room and sit on the throne. However, when attempting to interact with a hallucinated drink of wine, the game crashed. We also printed the cumulative lengths of the commands to make sure that everything was working properly there.

In [7]:
import hw1_solution.action_castle as action_castle

game = action_castle.build_game()
game.give_hints = True
parser = GptParser(game, verbose=False)

You should do a play through of Action Castle, and save a trascript of your play to submit along with your homework solution.  I recommend trying several commands that don't work.  Also, be on the look out for "hallucinations" by GPT, which in this context are elaborations that it adds that aren't part of the underlying game.  Try to interact with hallucinated objects and see what happens.  Is the underlying game state maintained?

In [ ]:
import json
import datetime

def save_game_transcript(game, filename_prefix="game_transcript"):
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    filename = f"{filename_prefix}-{current_datetime}.json"
    with open(filename, 'w') as file:
        # Pretty-print and save the command history
        json.dump(game.parser.command_history, file, indent=4)
        print(f"Game transcript saved successfully to {filename}")
 

save_game_transcript(game, "game_transcript_with_GPT_descriptions")

Game transcript saved successfully to game_transcript_with_GPT_descriptions-2024-02-20_03-52.json


## Part 2: Intent Determination  

The keyword-based parser for our HW1 text adventure game could only match very specific commands.  We could win the fishing mini game with this sequence of commands: 
1. take pole
2. go out
3. south
4. catch fish with pole
5. eat fish

However, if the user tried some other very similar commands, the simple keyword based parser wouldn't understand the meaning of these commands.

1. pick up the fishing pole
2. head outside
3. go down to the fishing pond
4. go fishing with the pole
5. I'm starving so I swallow the fish whole 

Try these commands using your GptParser – it won't work properly without special handling.  In fact, you might get out of sync with the game state. 

In this part of the homework, we'll extend your GptParser to more flexibly match user commands.  For this part of the homework, we'll implement a new version of the `determine_intent`.

There are several possible ways that you could flexibly match a user's input. 

* You could try using the [emeddings API](https://platform.openai.com/docs/guides/embeddings) 
* You could ask GPT-4 to pick the closest matching output from a list that you provide. 
* You could fine-tune GPT-3 to perform the task using [the fine-tuning API](https://platform.openai.com/docs/guides/fine-tuning). 

For my solution,  picked the second one. To ensure that I returned 
exactly the ACTION_NAME and not some longer text, I had to carefully craft 
my instructions to GPT, and eventually I resorted to using a regular 
expression over GPT's output).   

In [11]:
import regex

class GptParser2(GptParser):
    def __init__(self, game, echo_commands=True, verbose=False):
        super().__init__(game, echo_commands, verbose)
        self.refresh_command_list()

    def extract_digit(self, text):
        return regex.findall(r"[-]?\d+", text)[0]

    def refresh_command_list(self):
        # Command descriptions is a dictionary that maps
        # action descriptions and aliases onto action names 
        command_descriptions = {}
        for _, action in self.actions.items():
            description = action.ACTION_DESCRIPTION
            if action.ACTION_ALIASES:
                description += " (can also be invoked with '{aliases}')".format(
                    aliases = "', '".join(action.ACTION_ALIASES)
                )
            action_name = action.ACTION_NAME
            if action_name:
                command_descriptions[description] = action_name
        
        self.command_descriptions = command_descriptions
        return self


    def determine_intent(self, command):
        """
        Given a user's command, this method returns the ACTION_NAME of
        the intended action.  
        """
        lst = [str(idx)+': '+str(i)+'\n' for idx, i in enumerate(self.command_descriptions.keys())]
        choices = ''.join(lst)
        prompt_beg = "You are matching an input text to the most similar action phrase. If you are unsure of the correct input, the action is most likely \"go\".\n\
Focus on matching the action to the verb in the command. Here are some examples of matching verbs:\n\
\"grab\", \"take\", \"pick up\" ==> \"get\"\n\
\"leave behind\", \"discard\", \"leave\", \"leg go\" ==> \"drop\"\n\
\"leave for\", \"head to\", \"head towards\" ==> \"go\"\n\
\"strike\", \"fight\", \"hit\" ==> \"attack\"\n\
\"inspect\", \"study\", \"look at\" ==> \"examine\"\n\
\"gift\", \"hand over\", \"pass\", \"present\" ==> \"give\"\
\"consume\", \"taste\" ==> \"eat.\"\
\"consume\", \"taste\", \"imbibe\", \"sip\" ==> \"drink\"\n\
\"sniff\" ==> \"smell\"\n\
\"pluck\", \"take\", \"pull\" ==> \"pick rose\"\n\
\"go fishing\" ==> \"catch fish\"\n\
\"illuminate\" ==> \"light\"\n\
\"say runes\", \"chant runes\" ==> \"read runes\"\n\
You must only return the single number whose corresponding text best matches the given command:\n"
        prompt_end = "The best match is number: "
        total_prompt = prompt_beg+choices+prompt_end
        
        client = OpenAI()
        response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
            "role": "system",
            "content": total_prompt
            },
            {
            "role": "user",
            "content": command
            },
        ],
        temperature=0,
        max_tokens=10,
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0
        )

        idx = self.extract_digit(response.choices[0].message.content)

        # print(total_prompt)
        # print("Chosen Command:", list(self.command_descriptions.values())[int(idx)])
        
        return list(self.command_descriptions.values())[int(idx)]
    

### Test your determine_intent method.  

Feel free to increase the number of test cases.  Describe your testing in the README that you submit for your homework. 

In [ ]:
game = action_castle.build_game()
parser = GptParser2(game, verbose=False)
game.set_parser(parser)
parser.refresh_command_list()
print(parser.actions.keys())

test_cases = {
    "pick up the fishing pole": "get",
    "head outside": "go",
    "travel to the fishing pond": "go",
    "go fishing with the pole": "catch fish",
    "I'm starving so I swallow the fish whole": "eat",
    "head to the fishing pond": "go",
    "look at the rose": "examine",
    "let go of branch": "drop",
    "hit the troll": "attack",
    "pick a rose from the branch": "pick rose",
    "drink the wine": "drink",
    "check inventory": "inventory",
    "sniff the rose": "smell rose",
    "hit the troll with a branch": "attack",
    "grab fishing pole": "get",
    "read runes": "read runes",
    "wear crown": "wear crown",

    "PRINCESS HEAD OUTSIDE": "go",
    "have the guard travel to the fishing pond": "go",
    "the troll goes to the courtyard": "go",
    "send the ghost up the stairs": "go",

    "cast a line with the pole": "catch fish",
    "the princess goes fishing with the pole": "catch fish",
    "have the ghost go fishing": "catch fish",
    "the troll decides to catch a fish": "catch fish",

    "I'm starving so I swallow the fish whole": "eat",
    "The princess eats a whole fish": "eat",
    "The troll eats the fish": "eat",

    "drink the wine": "drink",
    "the princess sips the wine": "drink",

    "the player smacks the troll": "attack",
    "hit the guard with a branch": "attack",
    "the princess attacks the guard": "attack",

    "give the troll the fish": "give",
    "give the troll the fish": "give",
    "feed the fish to the troll": "give",
    "feed the fish to the troll": "give",
    
    "princess pick rose": "pick rose",
    "pluck the rose": "pick rose",
    "ghost takes a rose": "pick rose",

    "princess smell rose": "smell rose",
    "sniff the rose": "smell rose",

    "take a seat on the throne": "sit on throne",
    "princess sit on the throne": "sit on throne",
    
    "drop the crown": "drop",
    "have the princess let go of the key": "drop",
    "GHOST leaves the candle": "drop",

    "pick up the fishing pole": "get",
    "pick up the crown": "get",
    "have the princess take the key": "get",
    "GHOST grabs the fishing pole": "get",

    "examine the candle": "examine",
    "the princess looks at the crown": "examine",

    "check inventory": "inventory",
    "look over all items in possession": "inventory",
    "the princess's items": "inventory",

    "unlock door": "unlock door",
    "princess unlock door using the key": "unlock door",
    "ghost open door with key": "unlock door",

    "propose to the princess": "propose",
    "ask for the princess's hand in marriage": "propose",
    "the princess asks the guard to marry her": "propose",

    "light the candle": "light",
    "the guard lights the lamp": "light",

    "read the runes off of the candle": "read runes",
    "the guard reads the runes": "read runes",

    "place crown on head": "wear crown",
    "the princess puts on the crown": "wear crown",
}

correct = 0
total = 0
for command, action_name in test_cases.items():
    print(">", command)
    predicted_action = parser.determine_intent(command)
    total += 1
    if predicted_action == action_name:
        print(f"Correct: determine_intent predicted {predicted_action}")
        correct += 1
    else:
        print(f"Incorrect: determine_intent predicted {predicted_action} but it should have returned {action_name}")
print(f"{correct} out of {total} were correctly predicted.")

dict_keys(['sequence', 'attack', 'catch fish', 'describe', 'drink', 'drop', 'eat', 'examine', 'get', 'give', 'go', 'inventory', 'light', 'pick rose', 'quit', 'smell rose', 'unlock door', 'read runes', 'propose', 'wear crown', 'sit on throne'])
> pick up the fishing pole
Correct: determine_intent predicted get
> head outside
Correct: determine_intent predicted go
> travel to the fishing pond
Correct: determine_intent predicted go
> go fishing with the pole
Correct: determine_intent predicted catch fish
> I'm starving so I swallow the fish whole
Correct: determine_intent predicted eat
> head to the fishing pond
Correct: determine_intent predicted go
> look at the rose
Correct: determine_intent predicted examine
> let go of branch
Correct: determine_intent predicted drop
> hit the troll
Correct: determine_intent predicted attack
> pick a rose from the branch
Correct: determine_intent predicted pick rose
> drink the wine
Correct: determine_intent predicted drink
> check inventory
Correct: 

## Part 3: Argument Matching

After we've determined which of the Actions the user wants to perform, we need to flexibly match its arguments. If I user says "get the fishing rod", we want to execute the game command "get pole" even though the user didn't use the word "pole".

In this part, you should use GPT to implement the parser commands  `get_character`, `match_item` and `get_direction`.


In [12]:

from text_adventure_games.things.characters import Character
from text_adventure_games.things.items import Item
from text_adventure_games.things.locations import Location
import regex


class GptParser3(GptParser2):
    def __init__(self, game, echo_commands=True, verbose=False, model='gpt-4'):
        super().__init__(game, echo_commands, verbose)
        self.model = model

    def extract_digit(self, text):
        return regex.findall(r"[-]?\d+", text)[0]
    
    def get_character(
        self, command: str, hint: str = None, split_words=None, position=None
    ) -> Character:
        """
        This method tries to match a character's name in the command.
        If no names are matched, it defaults to `game.player`. 
        Args:
            hint: A hint about the role of character we're looking for 
                  (e.g. "giver" or "recipent")
            split_words: not needed for our GptParser
            position: not needed for our GptParser
        """

        system_prompt = "Given a command, return the character who can be described as: \"{h}\". ".format(h=hint)
        system_prompt += "Unless specified, assume \"0: The player\" performs all actions.\nChoose from the following characters:\n"
        
        # Create an enumerated dict of the characters in the game
        chars = {i: c for i, c in enumerate(list(self.game.characters))}
        
        # Format the characters into a list structure for the system prompt
        system_prompt += "{c}".format(c='\n'.join([str(i)+": "+str(c) for i, c in chars.items()]))

        system_prompt += "\nYou must only return the single number whose corresponding character is performing the action.\n\
If no command is given, return \"0: The player\""

        client = OpenAI()

        response = client.chat.completions.create(
            model = self.model,
            messages=[
            {
            "role": "system",
            "content": system_prompt
            },
            {
            "role": "user",
            "content": "Command: {c}\nThe best character match is number: ".format(c=command)
            },
            ],
            temperature=0,
            max_tokens=10,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )

        # Will probably need to do some parsing of the output here
        char_idx = response.choices[0].message.content
        try: 
            char_idx = self.extract_digit(char_idx)
            char_idx = int(char_idx)
        except Exception as e:
            print("Couldn't match the following response to a number:")
            print(e)

        # print(f"GPTParse selected character: {char_idx}")
        if char_idx not in chars:
            print(f"no player with id {char_idx} in {str(chars)}")
            return self.game.player
        else:
            name = chars[char_idx]
            return self.game.characters[name]

    def match_item(
        self, command: str, item_dict: dict[str, Item], hint: str = None
    ) -> Item:
        """
        Check whether the name any of the items in this dictionary match the
        command. If so, return Item, else return None.

        Args:
            item_dict: A map from item names to Items (could be a player's 
                       inventory or the items at a location)
            hint: what kind of item we're looking for
        """

        items_str = ", ".join(list(item_dict.keys()))
        system_prompt = "Given a command and a list of items, return the item that is best described as: \"{h}\".\nChoose from the following items:\n".format(h=hint)
        items = {i: it for i, it in enumerate(list(item_dict.keys()))}
        system_prompt += "{c}".format(c=''.join([str(i)+": "+str(item)+"\n" for i, item in items.items()]))
        system_prompt += """You must only return the single number whose corresponding item best matches the given command. \
If there are no good matches, return '-999'\n"""
        
        # print("Item system prompt: ", system_prompt)
        client = OpenAI()

        response = client.chat.completions.create(
            model = self.model,
            messages=[
            {
            "role": "system",
            "content": system_prompt
            },
            {
            "role": "user",
            "content": "Command: {c}\nItems: {i}\nThe item number that best matches the description \"{h}\" is: ".format(c=command, i=items_str, h=hint)
            },
            ],
            temperature=0,
            max_tokens=10,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )

        item_idx = response.choices[0].message.content
        try:
            item_idx = self.extract_digit(item_idx)
            item_idx = int(item_idx)
        except Exception as e:
            print(e)

        # print("Prompt:")
        # print(system_prompt)
        # print("Command: {c}\nItems: {i}\nThe item number that best matches the description \"{h}\" is: ".format(c=command, i=items_str, h=hint))
        # print(f"GPTParse selected item: {item_idx}")
        if item_idx == -999:
            return None
        elif item_idx in items:
            name = items[item_idx]
            return item_dict[name]
        else:
            print(f'Item index {item_idx} not found in {str(items)}')

    def get_direction(self, command: str, location: Location = None) -> str:
        """
        Return the direction from `location.connections` which the player
        wants to travel to.
        """
        dirs = list(location.connections.keys())
        names = [loc.name for loc in location.connections.values()]
        connections = {i: dl for i, dl in enumerate(zip(dirs, names))}
        # print('Found connections: ', connections)

        system_prompt = """
        You must select the direction that best matches the description given in a command.
        The possible directions to choose are:\n
        """
        
        system_prompt += "\n" + "{c}".format(c=''.join([str(i)+": "+str(d)+" or "+str(l)+"\n" for i, (d, l) in connections.items()]))
        
        system_prompt += """\nYou must only return the single number whose corresponding direction best matches the given command.
            If there are no good matches, return '-999'\n"""

        # print("Direction system prompt: ", system_prompt)

        client = OpenAI()

        response = client.chat.completions.create(
            model = self.model,
            messages= [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": "Command: {c}\n  The best direction match is number:  ".format(c=command)
                }
            ],
            temperature=0,
            max_tokens=100,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )

        dir_idx = response.choices[0].message.content
        try:
            dir_idx = self.extract_digit(dir_idx)
            dir_idx = int(dir_idx)
        except Exception as e:
            print(e)
        # print(f"GPTParse selected direction: {dir_idx}")

        if dir_idx in connections:
            dir_name = connections[dir_idx][0]
            return dir_name
        else:
            print(f'direction id "{dir_idx}" not in location connections: {connections}')
            return None



Test feeding the troll.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
parser.refresh_command_list()
command = "feed the fish to the troll"
give_words = ["give"]
giver = parser.get_character(command, hint="giver", split_words=give_words, position="before").name
recipient = parser.get_character(command, hint="recipient", split_words=give_words, position="after").name
print("Giver:", giver)
print("Recipient:", recipient)

Giver: The player
Recipient: troll


Test proposing to the princess.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
parser.refresh_command_list()
command = "propose to princess"
keyword = "propose"
(before_keyword, after_keyword) = command.split(keyword)
proposer = parser.get_character(before_keyword, hint="proposer")
propositioned = parser.get_character(after_keyword, hint="propositioned")
print("Proposer:", proposer.name)
print("Propositioned:", propositioned.name)

Proposer: The player
Propositioned: The player


Test attacking the guard with the branch.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
command = "smack the guard with the branch"
give_words = ["attack", "hit"]
attacker = parser.get_character(command, hint="attacker", split_words=give_words, position="before").name
victim = parser.get_character(command, hint="victim", split_words=give_words, position="after").name
print("Attacker:", attacker)
print("Victim:", victim)

Attacker: The player
Victim: guard


Test reading the runes.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
item_dict = {"candle": "candle", "lamp": "lamp",  "key": "key", "crown": "crown"}
game.set_parser(parser)
command = "candle"
reader = parser.get_character(command)
item = parser.match_item(command, item_dict, hint="candle")
print("Reader:", reader.name)
print("Item:", item)

Reader: The player
Item: candle


Test wearing the crown.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
item_dict = {"candle": "candle", "lamp": "lamp",  "key": "key", "crown": "crown"}
game.set_parser(parser)
command = "crown"
wearer = parser.get_character(command)
item = parser.match_item(command, item_dict, hint="crown")
print(wearer.name, item)

The player crown


Test sitting on the throne.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
item_dict = {"candle": "candle", "lamp": "lamp",  "key": "key", "crown": "crown", "throne":"throne"}
game.set_parser(parser)
command = "throne"
sitter = parser.get_character(command)
item = parser.match_item(command, item_dict, hint="throne")
print("Person:", sitter.name)
print("Item:", item)

Person: The player
Item: throne


Test unlocking the door.

In [ ]:
game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
item_dict = {"candle": "candle", "lamp": "lamp",  "key": "key", "crown": "crown", "throne":"throne", "door": "door"}
game.set_parser(parser)
command = "unlock the door"
unlocker = parser.get_character(command)
key = parser.match_item("key", item_dict, hint="key")
door = parser.match_item("door", item_dict, hint="door")
print("Person:", unlocker.name)
print("Item being unlocked:", door)
print("Item used to unlock door:", key)

Person: The player
Item being unlocked: door
Item used to unlock door: key


### Test your character_name method.

In [ ]:
game = action_castle.build_game()
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
parser.refresh_command_list()

test_cases = {
    ("head outside", "traveler"): "The player",
    ("head to the fishing pond", "traveler"): "The player",
    ("travel to the fishing pond", "traveler"): "The player",
    ("PRINCESS HEAD OUTSIDE", "traveler"): "princess",
    ("have the guard travel to the fishing pond", "traveler"): "guard",
    ("the troll goes to the courtyard", "traveler"): "troll",
    ("send the ghost up the stairs", "traveler"): "ghost",

    ("cast a line with the pole", "fisherman"): "The player",
    ("the princess goes fishing with the pole", "fisherman"): "princess",
    ("have the ghost go fishing", "fisherman"): "ghost",
    ("the troll decides to catch a fish", "fisherman"): "troll",

    ("I'm starving so I swallow the fish whole", "eater"): "The player",
    ("The princess eats a whole fish", "eater"): "princess",
    ("The troll eats the fish", "eater"): "troll",

    ("drink the wine", "drinker"): "The player",
    ("the princess sips the wine", "drinker"): "princess",

    ("the player smacks the troll", "attacker"): "The player",
    ("the player smacks the troll", "victim"): "troll",
    ("hit the guard with a branch", "attacker"): "The player",
    ("hit the guard with a branch", "victim"): "guard",
    ("the princess attacks the guard", "attacker"): "princess",
    ("the princess attacks the guard", "victim"): "guard",

    ("give the troll the fish", "giver"): "The player",
    ("give the troll the fish", "recipient"): "troll",
    ("feed the fish to the troll", "giver"): "The player",
    ("feed the fish to the troll", "recipient"): "troll",
    
    ("princess pick rose", "picker"): "princess",
    ("pluck the rose", "picker"): "The player",
    ("ghost takes a rose", "picker"): "ghost",

    ("princess smell rose", "smeller"): "princess",
    ("sniff the rose", "smeller"): "The player",

    ("take a seat on the throne", "sitter"): "The player",
    ("princess sit on the throne", "sitter"): "princess",
    
    ("drop the crown", "wants to drop something"): "The player",
    ("have the princess let go of to the key", "wants to drop something"): "princess",
    ("GHOST leaves the candle", "wants to drop something"): "ghost",

    ("pick up the fishing pole", "wants to get something"): "The player",
    ("pick up the crown", "wants to get something"): "The player",
    ("have the princess take the key", "wants to get something"): "princess",
    ("GHOST grabs the fishing pole", "wants to get something"): "ghost",

    ("examine the candle", "looker"): "The player",
    ("the princess looks at the crown", "looker"): "princess",

    ("check inventory", "person whose items are being enumerated"): "The player",
    ("look over items", "person whose items are being enumerated"): "The player",
    ("the princess's items", "person whose items are being enumerated"): "princess",
    ("look over the troll's items", "person whose items are being enumerated"): "troll",

    ("unlock door", "person unlocking thing"): "The player",
    ("princess unlock door using the key", "person unlocking thing"): "princess",
    ("ghost open door with key", "person unlocking thing"): "ghost",
    
}

correct = 0
total = 0
for command, character_name in test_cases.items():
    print(">", command)
    predicted_character = parser.get_character(command=command[0], hint=command[1]).name
    total += 1
    if predicted_character == character_name:
        print(f"Correct: character_name predicted {predicted_character}")
        correct += 1
    else:
        print(f"Incorrect: character_name predicted {predicted_character} but it should have returned {character_name}")
print(f"{correct} out of {total} were correctly predicted.")

> ('head outside', 'traveler')
Correct: character_name predicted The player
> ('head to the fishing pond', 'traveler')
Correct: character_name predicted The player
> ('travel to the fishing pond', 'traveler')
Correct: character_name predicted The player
> ('PRINCESS HEAD OUTSIDE', 'traveler')
Correct: character_name predicted princess
> ('have the guard travel to the fishing pond', 'traveler')
Correct: character_name predicted guard
> ('the troll goes to the courtyard', 'traveler')
Correct: character_name predicted troll
> ('send the ghost up the stairs', 'traveler')
Correct: character_name predicted ghost
> ('cast a line with the pole', 'fisherman')
Correct: character_name predicted The player
> ('the princess goes fishing with the pole', 'fisherman')
Correct: character_name predicted princess
> ('have the ghost go fishing', 'fisherman')
Correct: character_name predicted ghost
> ('the troll decides to catch a fish', 'fisherman')
Correct: character_name predicted troll
> ("I'm starving

### Test your match_item method.

In [ ]:
game = action_castle.build_game()
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
parser.refresh_command_list()

items_dict = {"candle": "candle", "lamp": "lamp", "key": "key", "crown": "crown", "throne":"throne", "door": "door",
             "rose": "rose", "wine": "wine", "water": "water", "sword": "sword", "branch": "branch", "key": "key",
             "pole": "pole", "fish": "fish", "rosebush": "rosebush"}

test_cases = {
    ("cast a line with the pole", "fishing pole"): "pole",
    ("the princess goes fishing with the pole", "fishing pole"): "pole",
    ("have the ghost go fishing with the pole", "fishing pole"): "pole",
    ("the troll decides to catch a fish using the pole", "fishing pole"): "pole",

    ("I'm starving so I swallow the fish whole", "food"): "fish",
    ("The princess eats a whole fish", "food"): "fish",
    ("The troll eats the fish", "food"): "fish",

    ("the player smacks the troll using a sword", "weapon"): "sword",
    ("hit the guard with a branch", "weapon"): "branch",
    ("the princess attacks the guard with a sword", "weapon"): "sword",

    ("hand the rose to the princess", "item being given"): "rose",
    ("princess give the crown to The player", "item being given"): "crown",
    ("feed the fish to the troll", "item being given"): "fish",
    ("give the fish to the troll to eat", "item being given"): "fish",

    ("sniff the rose", "rose"): "rose",
    ("princess smell rose", "rose"): "rose",

    ("princess pick a rose", "rosebush"): "rosebush",
    ("pluck the flower", "rosebush"): "rosebush",
    ("ghost takes a rose from the rosebush", "rosebush"): "rosebush",

    ("take a seat on the throne", "throne"): "throne",
    ("princess sit on the throne", "throne"): "throne",

    ("drink the wine", "drink"): "wine",
    ("the princess sips the water", "drink"): "water",
    
    ("drop the crown", "thing being dropped"): "crown",
    ("have the princess let go of to the key", "thing being dropped"): "key",
    ("GHOST leaves the candle", "thing being dropped"): "candle",

    ("pick up the fishing pole", "thing to get"): "pole",
    ("pick up the crown", "thing to get"): "crown",
    ("have the princess take the key", "thing to get"): "key",
    ("GHOST grabs the fishing pole", "thing to get"): "pole",

    ("unlock the door with the key", "door"): "door",
    ("unlock the door with the key", "key"): "key",
    ("the princess uses the key to open the door", "door"): "door",
    ("the princess uses the key to open the door", "key"): "key",

    ("examine the candle", "thing being looked at"): "candle",
    ("the princess looks at the crown", "thing being looked at"): "crown",
}

correct = 0
total = 0
for command, item_name in test_cases.items():
    print(">", command)
    predicted_item = parser.match_item(command=command[0], item_dict=items_dict, hint=command[1])
    total += 1
    if predicted_item == item_name:
        print(f"Correct: item_name predicted {predicted_item}")
        correct += 1
    else:
        print(f"Incorrect: item_name predicted {predicted_item} but it should have returned {item_name}")
print(f"{correct} out of {total} were correctly predicted.")

> ('cast a line with the pole', 'fishing pole')
Correct: item_name predicted pole
> ('the princess goes fishing with the pole', 'fishing pole')
Correct: item_name predicted pole
> ('have the ghost go fishing with the pole', 'fishing pole')
Correct: item_name predicted pole
> ('the troll decides to catch a fish using the pole', 'fishing pole')
Correct: item_name predicted pole
> ("I'm starving so I swallow the fish whole", 'food')
Correct: item_name predicted fish
> ('The princess eats a whole fish', 'food')
Correct: item_name predicted fish
> ('The troll eats the fish', 'food')
Correct: item_name predicted fish
> ('the player smacks the troll using a sword', 'weapon')
Correct: item_name predicted sword
> ('hit the guard with a branch', 'weapon')
Correct: item_name predicted branch
> ('the princess attacks the guard with a sword', 'weapon')
Correct: item_name predicted sword
> ('hand the rose to the princess', 'item being given')
Correct: item_name predicted rose
> ('princess give the c

### Test your get_direction method.

In [ ]:
game = action_castle.build_game()
parser = GptParser2(game, verbose=False)
game.set_parser(parser)
parser.refresh_command_list()

home = things.Location(
    "Home",
    "You are standing at home."
)
loc_in = things.Location(
    "In",
    "In"
)
loc_out = things.Location(
    "Out",
    "Out"
)
loc_down = things.Location(
    "Down",
    "Down"
)
loc_up = things.Location(
    "Up",
    "Up"
)
loc_east = things.Location(
    "East",
    "East"
)
loc_west = things.Location(
    "West",
    "West"
)
loc_north = things.Location(
    "North",
    "North"
)
loc_south = things.Location(
    "South",
    "South"
)
home.add_connection("in", loc_in)
home.add_connection("out", loc_out)
home.add_connection("up", loc_up)
home.add_connection("down", loc_down)
home.add_connection("north", loc_north)
home.add_connection("south", loc_south)
home.add_connection("east", loc_east)
home.add_connection("west", loc_west)

test_cases = {
    "head north": "north",
    "walk out": "out",
    "go in": "in",
    "head out": "out",
    "climb up the tree": "up",
    "descend down the stairs": "down",
    "travel east": "east",
    "walk west": "west",
    "run north": "north",
    "fly south": "south",
}

correct = 0
total = 0
for command, direction_name in test_cases.items():
    print(">", command)
    predicted_direction = parser.get_direction(command, location=home)
    total += 1
    if predicted_direction == direction_name:
        print(f"Correct: determine_intent predicted {predicted_direction}")
        correct += 1
    else:
        print(f"Incorrect: determine_intent predicted {predicted_direction} but it should have returned {direction_name}")
print(f"{correct} out of {total} were correctly predicted.")

> head north
Correct: determine_intent predicted north
> walk out
Correct: determine_intent predicted out
> go in
Correct: determine_intent predicted in
> head out
Correct: determine_intent predicted out
> climb up the tree
Correct: determine_intent predicted up
> descend down the stairs
Correct: determine_intent predicted down
> travel east
Correct: determine_intent predicted east
> walk west
Correct: determine_intent predicted west
> run north
Correct: determine_intent predicted north
> fly south
Correct: determine_intent predicted south
10 out of 10 were correctly predicted.


Next, do another playthrough of the game and try to use only commands that do not correspond to the ones we had in HW1.  Save your trascript and upload it.

This is a run through the game using GPT-4 to determine the actions, characters, items, and directions. It uses GPT-3.5-turbo to create the game descriptions to save money. This version also prints the command history for debugging.

In [ ]:
import hw1_solution.action_castle as action_castle
import openai

game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
parser.refresh_command_list()
try:
    game.game_loop()
except openai.APIConnectionError:
    print("Could not connect to OpenAI API")

> look
Passed Command History:
[{'role': 'user', 'content': 'look'}, {'role': 'assistant', 'content': 'You are standing in a small cottage.\nExits:\nOut to Garden Path\n\nYou see:\n * a fishing pole\n\n'}]
The room is dimly lit by a flickering candle, casting eerie shadows on the
walls. A musty smell hangs in the air, mingling with the scent of damp earth. A
single window lets in a sliver of moonlight, illuminating a dusty bookshelf in
the corner. A tattered rug lies on the creaking wooden floor, and a moth-eaten
armchair sits by the fireplace, its embers glowing faintly. The only exit is a
heavy oak door leading out to the garden path.

You see:
 * a fishing pole

> Grab fishing pole, head out, go to fishing pond, go fishing with pole, head to the garden, pluck a rose, sniff the rose, head to the winding path, climb the branches, take branch, descend the tree, walk to the bridge, feed the fish to the troll, go into the courtyard, smack the guard with the branch, collect key, travel in

In [ ]:

save_game_transcript(game, "game_transcript_with_flexible_commands")

Game transcript saved successfully to game_transcript_with_flexible_commands-2024-02-19_18-29.json


## Optional Extension: Comparing Approaches to Intent Determination 

If you're working in a group larger than a pair, then you should pick one optional extension for each group member above 2.  One extension that you could do is to compare several approaches to intent determination, and quanify how good each approach is on an extended test set that you put together.



## Optional Extension: Image creation 

In addition to creating evocative text with the OpenAI API, you can also use it to create images with DALLE-3.  Below are some examples.  If you do this extension, you should pre-generate images for each of the locations in your game, and save them.  Then you can display them when a character moves into a location.  You can generate several and choose your favorites.  In your writeup, describe what prompts you used, and what you did to try to keep a consistent image style throughout the game.

In [ ]:


img_prompts = [
"""
Style: Text Adventure Game like Zork.

You are standing in a small cottage.
Exits:
Out to Garden Path

You see:
 * a fishing pole
 * a poisonous potion

""",

"""
Style: Jane Austen

As you gaze around, you find yourself ensconced within a humble yet enchanting little cottage, stowed away like a precious secret in the heart of the country. The air is redolent with the comforting aroma of wood smoke and freshly baked bread, warming the cockles of one's heart.

The cottage breathes and lives through its evidence of mundane life: a neglected fishing pole that must have had stories old and new twined around its line, resting against the wall near the window, almost in anticipation of the next adventure. On a sturdy oak table below the window, stands a mysteriously dark vial, the poisonous potion it houses shimmering under the faint candlelight.

At your disposal, my dear adventurer, lies the exit – a door, simple yet inviting, promising the rustic charm of a garden path beyond its threshold. Whatever path you choose, remember, curiosity tempered with caution is the true calling of an adventurer. Through the looking glass, my dear, through the looking glass.
""",


"""
Style: Edgar Allen Poe

Encircled by the foreboding veil of twilight, you find yourself within the confines of a quaint, yet somewhat desolate cottage. Tattered curtains tremble as a chilly breeze invades the space, and the lowly lit fire casts a lurid flicker across the room, spawning grotesque shadows on the bare stone walls. 

Options lie before you:
Paths lead you back towards the sinuous charm of the whispering garden from whence you came.

Upon your survey, objects of intrigue claw out from the gloom:
* A fishing pole, a skeleton of erstwhile merriment, it lay unused and embraced by layers of dust. 
* A pulsating vial is also there, a phantom-like glow emanates from it, the liquid within as deadly as the most lethal of venoms, tinging the air with an atmosphere of potent danger and allure.
""",



"""
Style: William Shakespeare 

Thou dost find thyself within the humble confines of a modest cottage, seeming born from the earth herself. 'Tis a space small in size, yet grand in its cosiness and peace, imbued with the warmth of bygone days.

Exit there are but one:
Yonder lies the verdant garden path, serenading with the soft whispers of the zephyr.

Present sights before thee are:
* A fishing pole, lean and patient as if anticipating the secret dance of the river's denizens.
* A draught of venomous potion, dancing with dread darkness and fey mystery.
""",

"""
Style: Donald Trump

You're in this beautiful, yuge cottage folks, it's just perfect. We've got the most beautiful cottages, I'll tell you that.

Look, here's how you get around: You've got this charming little garden path. And what a garden path folks, it’s like nothing you've ever seen before, just splendid.

Now look here, you see that? You've got two things in here:

1. A fishing pole. And it's not just any fishing pole. It's a totally first-class, top-notch, best you've ever seen fishing pole. 

2. Watch out though! There’s this totally toxic, a disastrous potion. Trust me it's no good, folks. It's just poison in a pretty bottle. I wouldn't touch it if I were you. It’s a disaster!
""",

"""
Style: Young Adult Dystopian Fiction

You find yourself in a ramshackle one-room cottage, abandoned long ago by anyone who once entertained life's ordinary pleasures. It is dusty and forlorn yet somehow undeniably cozy. The late afternoon sunlight seeps weakly through the moth-eaten curtains, painting patterns on the worn wooden floorboards that hint of forgotten laughter and secrets whispered in the night. 

To the north, the door, barely hanging on its hinges, leads to what was once a charming garden path, now reclaimed by the unyielding grasp of nature; brambles and twisted vines run rampant, marking the passage of time that mankind forgot.

Around you, a few relics hold out stubbornly against the annihilation of civilization. A forsaken fishing pole rests against the shadowy corner, whispering tales of tranquil days, while a vial of radiant, emerald-green liquid - a poisonous potion, no doubt concocted in the crucible of man's unquenchable thirst for dominance - is more ominous, sitting quietly on the rustic table like a ticking time bomb.
""",


"""
Style: Pulp Fiction and Samuel L. Jackson

Look around and what do you see, motherfucker? We're in this tiny-ass cottage, feels like you could touch each wall if you stretched your arms wide. It's got that cozy vibe going on, but don't let it fool you, nothing's ever what it seems.

You've got two ways out. One is the way you came in...a garden path. Tranquil as shit, green, just like you'd imagine. 

Now here's the fun part. A goddamn fishing pole is sitting there, all cool, waiting to be picked up. You know when you see a fishing pole that shit's about to get interesting. 

And what's this? A damn poisonous potion. Don't ask me why that's there, I'm just the Samuel L. Jackson flavoured voice in your head. You decide what to do with it. But whatever it is, you'd better reckon it fast, motherfucker.
""",

]

